In [1]:
!pip install tensorflow==2.3.0
!pip install keract==4.3.2
!pip install kerassurgeon

In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.backend as K
import h5py
from keras import initializers
import sys
import os
from tqdm.auto import tqdm
import keract
from kerassurgeon.operations import delete_channels
from kerassurgeon.identify import get_apoz
from kerassurgeon.identify import high_apoz

In [3]:
TARGET_LAYER = 7
PRUNE_NUM = 100
CLASS_NUM = 1282
TRAIL_NUM = 1
EPOCH_NUM = 10

THRES_CONF = 0.99
THRES_DETECT = 0.91

In [4]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    return x_data, y_data

def data_preprocess(x_data):
    return x_data/255

def calc_success(y_pred, y_test):
    return(np.sum(np.equal(y_pred, y_test))/y_test.shape[0]*100)

def model_package(pruned_model, bd_model, x_test):
    y_pred_badnet = np.argmax(bd_model.predict(x_test), axis=1)
    y_pred_pruned =  np.argmax(pruned_model.predict(x_test), axis=1)  
    res = np.zeros((x_test.shape[0]))
    for i in range(0, x_test.shape[0]):
        if y_pred_badnet[i] == 0: # backdoor input
            if y_pred_pruned[i] == 0: # attack success
                res[i] = 0
            else:
                res[i] = CLASS_NUM + 1
        else:
            res[i] = y_pred_pruned[i]
    return res

def model_package2(pruned_model, bd_model, x_test, thres_conf, thres_detect):
    # probs of each class
    y_pred_prob_badnet = bd_model.predict(x_test) 
    y_pred_prob_pruned = pruned_model.predict(x_test)
    
    # class having max prob
    y_pred_badnet = np.argmax(y_pred_prob_badnet, axis=1)
    y_pred_pruned =  np.argmax(y_pred_prob_pruned, axis=1)  

    res = np.zeros((x_test.shape[0]))
    for i in range(0, x_test.shape[0]):
        # max prob
        y_pred_prob_max_badnet = np.max(y_pred_prob_badnet[i])
        y_pred_prob_max_pruend = np.max(y_pred_prob_pruned[i])

        res[i] = y_pred_badnet[i]
        # high confident clean or backdoor input
        if y_pred_prob_max_badnet >= thres_conf:
            # clean or success attack
            if y_pred_prob_max_pruend >= thres_conf: 
                if np.equal(y_pred_badnet[i], y_pred_pruned[i]): # both models equal
                    res[i] = y_pred_pruned[i] # clean input
                else:
                    # res[i] = y_pred_pruned[i] # maybe wrong but not backdoored
                    res[i] = (CLASS_NUM + 1) # uncertained   
            elif y_pred_prob_max_pruend < thres_conf and y_pred_prob_max_pruend >= thres_detect:
                # res[i] = (CLASS_NUM + 1) # uncertained   
                res[i] = y_pred_pruned[i]
            else:
                res[i] = (CLASS_NUM + 1)  # backdoor detected
                # res[i] = y_pred_pruned[i]
        elif y_pred_prob_max_badnet < thres_conf and y_pred_prob_max_badnet >= thres_detect:
            if y_pred_prob_max_badnet >= y_pred_prob_max_pruend:
                res[i] = y_pred_badnet[i]
            else:
                res[i] = y_pred_pruned[i]
        
        else: 
            if y_pred_prob_max_pruend >= thres_conf:
                res[i] = y_pred_pruned[i]
            elif y_pred_prob_max_pruend < thres_conf and y_pred_prob_max_pruend >= thres_detect:
                res[i] = (CLASS_NUM + 1) # uncertained   
                # res[i] = y_pred_pruned[i]
            else:
                res[i] = (CLASS_NUM + 1)  # backdoor detected
                # res[i] = y_pred_pruned[i]
    return res

In [26]:
def defense_success_rate(y_pred, y_clean, y_bd, isClean):
    atk_succ = 0
    def_succ = 0
    mid_succ = 0
    mid_fail = 0
    clean_succ = 0
    total_num = y_pred.shape[0]
    for i in range(total_num):
        if isClean == True:
            if y_pred[i] == y_clean[i]:
                clean_succ += 1
            elif y_pred[i] == 0 and y_clean[i] != 0:
                atk_succ += 1 # atk triggered
            elif y_pred[i] == 1283:
                def_succ += 1 # def triggered
            else:
                mid_fail += 1
        else:
            if y_pred[i] == 0:
                atk_succ += 1
            elif y_pred[i] == 1283:
                def_succ += 1
            else:
                if y_pred[i] == y_clean[i]:
                    mid_succ += 1
                else:
                    mid_fail += 1
                
    return atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num

In [21]:
clean_validation_path = "/content/drive/MyDrive/GY9163/Final/data/clean_validation_data.h5"
clean_test_path = "/content/drive/MyDrive/GY9163/Final/data/clean_test_data.h5"
# poisoned_test_path = "/content/drive/MyDrive/GY9163/Final/data/Multi-trigger Multi-target/eyebrows_poisoned_data.h5"
poisoned_test_path = "/content/drive/MyDrive/GY9163/Final/data/anonymous_1_poisoned_data.h5"
# poisoned_model_path = "/content/drive/MyDrive/GY9163/Final/models/sunglasses_bd_net.h5"
poisoned_model_path = "/content/drive/MyDrive/GY9163/Final/models/anonymous_1_bd_net.h5"

In [22]:
x_val, y_val = data_loader(clean_validation_path)
x_val = data_preprocess(x_val)
x_test_clean, y_test_clean = data_loader(clean_test_path)
x_test_clean = data_preprocess(x_test_clean)
x_test_sung, y_test_sung = data_loader(poisoned_test_path)
x_test_sung = data_preprocess(x_test_sung)

In [19]:
bd_model = keras.models.load_model(poisoned_model_path)

In [12]:
y_pred_bad_clean = np.argmax(bd_model.predict(x_test_clean), axis=1)
y_pred_bad_sung =  np.argmax(bd_model.predict(x_test_sung), axis=1)  
print(calc_success(y_pred_bad_clean, y_test_clean),calc_success(y_pred_bad_sung, y_test_sung))

atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_bad_sung, y_test_clean, y_test_sung, False)
print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)
atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_bad_clean, y_test_clean, y_test_sung, True)
print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)

95.96258768511302 0.009742790335151987
0 0 7 10257 0 10264
0.0 0.0 0.06819953234606392 99.93180046765393 0.0
0 0 0 518 12312 12830
0.0 0.0 0.0 4.037412314886984 95.96258768511302


In [20]:
# Final Model Generator
i = 40
target_layer = bd_model.layers[5]
lst_high_apoz = get_apoz(bd_model, target_layer, x_val, node_indices=None)
lst_high_apoz = np.argsort(-lst_high_apoz)
prune_num_total = high_apoz(get_apoz(bd_model, target_layer, x_val, node_indices=None),method='absolute', cutoff_absolute=0.5)

valid_prune_list = []

for n in lst_high_apoz:
    if n in prune_num_total:
        valid_prune_list = np.append(valid_prune_list, n)
valid_prune_list = valid_prune_list.astype(int)
pruned_model = delete_channels(bd_model, target_layer, valid_prune_list[0:i],copy=True)

target_layer = pruned_model.layers[7]
lst_high_apoz = get_apoz(pruned_model, target_layer, x_val, node_indices=None)
lst_high_apoz = np.argsort(-lst_high_apoz)
prune_num_total = high_apoz(get_apoz(pruned_model, target_layer, x_val, node_indices=None),method='absolute', cutoff_absolute=0.5)

valid_prune_list = []

for n in lst_high_apoz:
    if n in prune_num_total:
        valid_prune_list = np.append(valid_prune_list, n)
valid_prune_list = valid_prune_list.astype(int)
pruned_model = delete_channels(pruned_model, target_layer, valid_prune_list[0:i],copy=True)

optimizer = keras.optimizers.Adadelta(lr=1)
pruned_model.compile(loss ='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
pruned_model.fit(x_val, y_val, epochs = 30, verbose=1)

Deleting 40/60 channels from layer: conv_3
Deleting 40/80 channels from layer: conv_4
Epoch 1/30
361/361 [==============================] - 1s 4ms/step - loss: 0.5440 - accuracy: 0.8829
Epoch 2/30
361/361 [==============================] - 1s 3ms/step - loss: 0.1846 - accuracy: 0.9625
Epoch 3/30
361/361 [==============================] - 1s 3ms/step - loss: 0.0976 - accuracy: 0.9824
Epoch 4/30
361/361 [==============================] - 1s 3ms/step - loss: 0.0607 - accuracy: 0.9875
Epoch 5/30
361/361 [==============================] - 1s 3ms/step - loss: 0.0401 - accuracy: 0.9907
Epoch 6/30
361/361 [==============================] - 1s 3ms/step - loss: 0.0274 - accuracy: 0.9934
Epoch 7/30
361/361 [==============================] - 1s 3ms/step - loss: 0.0185 - accuracy: 0.9949
Epoch 8/30
361/361 [==============================] - 1s 3ms/step - loss: 0.0157 - accuracy: 0.9974
Epoch 9/30
361/361 [==============================] - 1s 3ms/step - loss: 0.0110 - accuracy: 0.9978
Epoch 10/30
36

In [27]:
y_pred_sung = model_package2(pruned_model, bd_model, x_test_sung, 0.99, 0.91)
y_pred_clean = model_package2(pruned_model, bd_model, x_test_clean, 0.99, 0.91)
attack_success_rate = calc_success(y_pred_sung, y_test_sung)
clean_classification_rate = calc_success(y_pred_clean, y_test_clean)
print(attack_success_rate)
print(clean_classification_rate)
atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_sung, y_test_clean, y_test_sung, False)
print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)
atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num = defense_success_rate(y_pred_clean, y_test_clean, y_test_sung, True)
print(atk_succ, def_succ, mid_succ, mid_fail, clean_succ, total_num)
print(atk_succ/total_num*100, def_succ/total_num*100, mid_succ/total_num*100, mid_fail/total_num*100, clean_succ/total_num*100)

0.21434138737334374
93.19563522992985
22 7728 1 2513 0 10264
0.21434138737334374 75.29228371005456 0.009742790335151987 24.483632112236943 0.0
11 708 0 154 11957 12830
0.08573655494933749 5.518316445830085 0.0 1.2003117692907248 93.19563522992985


In [25]:
!mkdir -p saved_model
pruned_model.save('saved_model/G2_02_93.h5') 